In [1]:
import pandas as pd
import spacy
import re
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin
from sklearn.metrics import classification_report

import sys
sys.path.append(r'../src')
from Preprocessing_data import Preprocessing_data as pre

In [2]:
nlp_cat = spacy.load('../train/sym/sym_model/model-best')

In [3]:
nlp_a = spacy.load('../train/subsym/subsym_a/subsym_model/model-best/')

In [4]:
nlp_b = spacy.load('../train/subsym/subsym_b/subsym_model/model-best/')

In [5]:
df = pd.read_csv('../../raw_data/df_full_banks.csv')

In [32]:
df = df.query('rating != "отозван"')

In [37]:
df = df.query('rating != "Отозван"')

In [11]:
df = df.rename(columns={'press_release_text': 'pr_txt'})

In [6]:
#df = df.rename(columns={'Категория': 'category', 'Уровень рейтинга': 'rating'})

In [13]:
df['clear_text'] = df['pr_txt'].apply(pre.drop_tail)

In [14]:
df['clear_text'] = df['clear_text'].apply(pre.clear)


In [15]:
df['clear_text'] = df['clear_text'].apply(pre.lemmatize)


In [16]:
df['clear_text'] = df['clear_text'].apply(pre.tokenize_it)

In [18]:
df['category'] = df['rating'].str.replace('+', '').str.replace('-', '')

C:\Users\wilte\AppData\Local\Temp\ipykernel_3560\3226637405.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category'] = df['rating'].str.replace('+', '').str.replace('-', '')


In [39]:
def get_prection_cat(text):
    doc = nlp_cat(text)
    scores = doc.cats
    if max(scores, key=scores.get) == 'A':
        doc = nlp_a(text)
        scores = doc.cats
        return max(scores, key=scores.get)
    elif max(scores, key=scores.get) == 'B':
        doc = nlp_b(text)
        scores = doc.cats
        return max(scores, key=scores.get)   

In [40]:
df['pred_rating'] = df['clear_text'].apply(get_prection_cat)

C:\Users\wilte\AppData\Local\Temp\ipykernel_3560\491850382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred_rating'] = df['clear_text'].apply(get_prection_cat)


In [41]:
df['pred_category'] = df['pred_rating'].str.replace('+', '').str.replace('-', '')

C:\Users\wilte\AppData\Local\Temp\ipykernel_3560\2885058114.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['pred_category'] = df['pred_rating'].str.replace('+', '').str.replace('-', '')
C:\Users\wilte\AppData\Local\Temp\ipykernel_3560\2885058114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred_category'] = df['pred_rating'].str.replace('+', '').str.replace('-', '')


In [42]:
f1_score(df['category'], df['pred_category'], average='weighted')

0.1493937342716477

In [43]:
f1_score(df['rating'], df['pred_rating'], average='weighted')

0.04767524938486512

In [44]:
f1_score(df['category'], df['pred_category'], average='weighted') * 0.35 + f1_score(df['rating'], df['pred_rating'], average='weighted') * 0.65

0.08327671909523902

In [45]:
print(classification_report(df['category'], df['pred_category']))

              precision    recall  f1-score   support

           A       0.26      0.20      0.23        46
          AA       0.33      0.04      0.08        23
         AAA       0.17      0.04      0.07        24
           B       0.00      0.00      0.00        44
          BB       0.35      0.11      0.17        64
         BBB       0.18      0.86      0.30        37

    accuracy                           0.21       238
   macro avg       0.22      0.21      0.14       238
weighted avg       0.22      0.21      0.15       238



c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

In [46]:
print(classification_report(df['rating'], df['pred_rating']))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00        13
          A+       0.50      0.13      0.21        15
          A-       0.08      0.11      0.09        18
          AA       0.33      0.08      0.12        13
         AA+       0.00      0.00      0.00         7
         AA-       0.00      0.00      0.00         3
         AAA       0.17      0.04      0.07        24
           B       0.00      0.00      0.00        16
          B+       0.00      0.00      0.00        20
          B-       0.00      0.00      0.00         8
          BB       0.10      0.14      0.12        14
         BB+       0.00      0.00      0.00        26
         BB-       0.00      0.00      0.00        24
         BBB       0.07      0.10      0.08        10
        BBB+       0.05      0.89      0.09         9
        BBB-       0.00      0.00      0.00        18

    accuracy                           0.07       238
   macro avg       0.08   

c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`